# Basic LSTMs

In [2]:
import sys
## To get helper functions from another folder
# sys.path.insert(0, '../') # if _helper_basics_ is in previous folder
now_i_am_at = 'home' # home dso test
if now_i_am_at=='home':
    sys.path.insert(0, 'E:/Leonard HDD/Dropbox/DSO/Tasks/')
elif now_i_am_at=='dso':
    sys.path.insert(0, 'D:/Dropbox/DSO/Tasks')

from _helper_basics_ import *

## Deep learning
# import tensorflow as tf
# import keras
# from keras.models import Model, Sequential
# from keras.layers import *
# from keras import optimizers
# import keras.backend as K

Using TensorFlow backend.


In [3]:
%matplotlib inline
plt.rcParams['savefig.dpi'] = 100

## Why need autoreload - https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

print('keras', keras.__version__)
print('tensorflow', tf.__version__)

keras 2.0.5
tensorflow 1.2.0


# One-to-Many LSTM

## Image Captioning
2 Versions, whether Image/Image_code as input

In [200]:
def set_X(Input_type, m,n,channel, num_feat_inp):
    if Input_type == 'image':
        ## Input : Image
        # (None, m, n, channel)
        #    m,n        : width,height of image
        #    channel    : B/W or Color
        m,n = 5,5
        channel = 1
        X = np.random.rand(1, m,n, channel)
        X = X.reshape(1, m,n, channel)
    elif Input_type == 'embedding':
        ## Input : Embedding from a ConvNet/etc
        # input for LSTMs must be three dimensional.
        # (None, num_feat_inp)
        #    num_feat_inp : lvl of embedding
        X = np.random.rand(1, num_feat_inp)
        X = X.reshape(1, num_feat_inp)
    return X
def set_Archi_1(Input_type, m,n,channel, num_feat_inp):
    if Input_type == 'image':
        ## Input is a image
        Inp = Input( shape=(m,n, channel),name = "Image" )
        ######################## Image_Code (ConvNet) #######################
        # x = ResNet50(weights='imagenet')(Inp)
        x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv2D_1')(Inp)
        x = Flatten()(x)
        x = Dense(num_feat_inp, name='Image_Code')(x)
        #####################################################################
    elif Input_type == 'embedding':
        Inp = Input( shape=(num_feat_inp,), name = "Image_Code" )
        x = Inp
    return Inp, x

Input_type = 'embedding' #'embedding' 'image'
X = set_X(Input_type, m,n,channel, num_feat_inp)
## Output : Caption
# (None, num_words, num_feat_out)
#    num_words    : num of words in a sentence.
#    num_feat_out : lvl of embedding
num_feat_inp = 10 # For Image Code/Embedding
num_words    = 3
num_feat_out = 2
y = np.random.rand(1, num_words, num_feat_out)
y = y.reshape(1, num_words, num_feat_out)

########################### Architecture ##################################
Inp, x = set_Archi_1(Input_type, m,n,channel, num_feat_inp)
########################### Recurrent Section ###########################
## RepeatVector to tally the number of words output.
x = RepeatVector(num_words, name='Repeat')(x)
x = LSTM(num_feat_out, name = "RNN_1", return_sequences=True)(x)
########################### Printing information ########################
model = Model(Inp, x)
model.summary()
# print_model_weights(model)
################################# Training ##############################
n_batch = 1
n_epoch = 1
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=1)
############################### Evaluation #############################
result = model.predict(X)
print('X:\n',X[0].T)
print('result:\n',result[0].T)
print('y:\n',y[0].T)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_Code (InputLayer)      (None, 10)                0         
_________________________________________________________________
Repeat (RepeatVector)        (None, 3, 10)             0         
_________________________________________________________________
RNN_1 (LSTM)                 (None, 3, 2)              104       
Total params: 104
Trainable params: 104
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
1/1 [==============================] - 3s - loss: 0.1880
X:
 [ 0.06110775  0.90476344  0.22743712  0.53567283  0.59140566  0.64479753
  0.63045467  0.7113344   0.90057832  0.80999393]
result:
 [[ 0.2347582   0.31728387  0.34880367]
 [ 0.04969562  0.08802228  0.11605316]]
y:
 [[ 0.06333702  0.78159723  0.58280625]
 [ 0.85212739  0.4986673   0.20912442]]


# Many-to-One LSTM for Sequence Classification 

## Sentiment Analysis

In [195]:
length = 5
seq = np.array([i/float(length) for i in range(length)])
class_type = np.array([0,1])
## Input 
# The input for LSTMs must be three dimensional.
# (None, length, feature)
#    None       : len(data)
#    length     : duration of speech,text
#    feature    : lvl of embedding
X = seq.reshape(1, length, 1)
## Output 
# (None, feature)
#    None       : len(data)
#    feature    : lvl of embedding
y = class_type.reshape(1, 2)

n_neurons = length
########################### Architecture ##################################
Inp = Input( shape=(5,1),name = "Input" )
x = LSTM(2, name = "RNN_1")(Inp) 
# x = Dense(5,name = "Output" )(x)
model = Model(Inp, x)
########################### Printing information ########################
model.summary()
# print_model_weights(model)
############################### Evaluation #############################
n_batch = 1
n_epoch = 1
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=1)
############################### Evaluation #############################
result = model.predict(X, batch_size=1, verbose=0)
print('X:\n',X[0].T)
print('result:\n',result[0].T)
print('y:\n',y[0].T)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 5, 1)              0         
_________________________________________________________________
RNN_1 (LSTM)                 (None, 2)                 32        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
1/1 [==============================] - 2s - loss: 0.7656
X:
 [[ 0.   0.2  0.4  0.6  0.8]]
result:
 [ 0.25404558 -0.20936279]
y:
 [0 1]


# Many-to-Many RNN

## Sequence Prediction
            Many(Input)  : [0,1,2,3]
            Many(Output) :          [4,5,6,7]
## Machine Translation
            Many(Input)  : ['I','want']
            Many(Output) :             ['我','要']

In [192]:
embedding     = 1
num_words_in  = 4
LSTM_neurons  = 10
num_words_out = 4
X = np.array([0,1,2,3]).reshape(1,num_words_in, embedding)
y = np.array([4,5,6,7]).reshape(1,num_words_out,embedding)
########################### Architecture ##################################
Inp = Input( shape=(num_words_in, embedding),name = "Input" )
x = LSTM(LSTM_neurons, name = "Encoder")(Inp)

x = RepeatVector(num_words_out, name='Repeat')(x)

x = LSTM(LSTM_neurons, name = "Decoder_1", return_sequences=True)(x)
x = LSTM(embedding, name = "Decoder_2", return_sequences=True)(x)
########################### Printing information ########################
model = Model(inputs=Inp, outputs=x)
model.summary()
# print_model_weights(model)

n_batch = 1
n_epoch = 1
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=1)
print('X:\n',X.reshape(num_words_in))
print('predict:\n', model.predict(X.reshape(1,num_words_in,embedding)).reshape(num_words_out).T)
print('y:\n',y.reshape(num_words_out))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 4, 1)              0         
_________________________________________________________________
Encoder (LSTM)               (None, 10)                480       
_________________________________________________________________
Repeat (RepeatVector)        (None, 4, 10)             0         
_________________________________________________________________
Decoder_1 (LSTM)             (None, 4, 10)             840       
_________________________________________________________________
Decoder_2 (LSTM)             (None, 4, 1)              48        
Total params: 1,368
Trainable params: 1,368
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
1/1 [==============================] - 5s - loss: 31.5729
X:
 [0 1 2 3]
predict:
 [-0.0019764  -0.00380387 -0.00473887 -0.00476281

## Completing Last Char
    Many(Input)  : ['h','e','l','l']
    Many(Output) :     ['e','l','l','o']

In [193]:
num_chars = 4
num_features = 1
num_neurons = 99
hello = np.array([0,1,2,3,4])
X = hello[:4].reshape(1,num_chars,num_features)
y = hello[1:].reshape(1,num_chars,num_features)

## This Architecture is assuming 1 input
n_neurons = 100
num_channel = 1 # this is just 1, when using conv layers this dimension is to fit filter=64, etc
########################### Architecture ##################################
Inp = Input( shape=( num_chars,num_features),name = "Input" )
## Intermediate layers
x = LSTM(num_neurons, name = "RNN_1", return_sequences=True)(Inp) 
# x = LSTM(num_neurons, name = "RNN_2", return_sequences=True)(x) 
# x = LSTM(n_neurons, name = "RNN_2")(x) 
## Output layer
x = LSTM(num_features, activation='linear', name = "Output", return_sequences=True)(x) 
model = Model(Inp, x)
########################### Printing information ########################
model.summary()
# print_model_weights(model)
################################# Training ##############################
n_batch = 1
n_epoch = 1
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=1)
################################ Evaluation #############################
result = model.predict(X)
print('result:\n',result[0].T)
print('y:\n',y[0].T)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 4, 1)              0         
_________________________________________________________________
RNN_1 (LSTM)                 (None, 4, 99)             39996     
_________________________________________________________________
Output (LSTM)                (None, 4, 1)              404       
Total params: 40,400
Trainable params: 40,400
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
1/1 [==============================] - 3s - loss: 7.4781
result:
 [[ 0.00098981  0.00435242  0.01138499  0.02222412]]
y:
 [[1 2 3 4]]


## Video Captioning
    Many(Input)  : ['frame_1','frame_2','frame_3','frame_4']
    Encode_Layer : [ 'code_1', 'code_2', 'code_3', 'code_4']
    Many(Output) :                                  ['I','eat','rice','.']

In [130]:
num_frames, m,n,channel = 5, 10,10, 3
num_words, num_features = 5, 2

In [137]:
Input_type = 'image' # 'embedding'

## EDIT : it doesnt seem to tally.

if Input_type == 'image':
    Inp = Input( shape=(m,n, channel),name = "Input" )
    x = Conv2D(64, (3,3), activation='relu', padding='same')(Inp)
    x = Flatten()(x)
    x = Dense(64, name='Conv_Mod_Embedding')(x)
    Conv_Model = Model(Inp ,x)
    ########################### Printing information ########################
#     Conv_Model.summary()
    # print_model_weights(Conv_Model)

    num_timestep = 5
    audio_input = Input(shape=(num_timestep, m,n, channel))
    encoded_frame_sequence = TimeDistributed(Conv_Model)(audio_input)  
    x = LSTM(2)(encoded_frame_sequence)  # the output will be a vector
    Video_Captioning_Model = Model(inputs=audio_input, outputs=x)


# elif Input_type == 'embedding':
#     Inp = Input( shape=(1,num_features),name = "Input" )
#     x = LSTM(num_features, name = "RNN_1")(Inp) 
#     Video_Captioning_Model = Model(Inp, x)

########################### Printing information ########################
Video_Captioning_Model.summary()
# print_model_weights(Video_Captioning_Model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 5, 10, 10, 3)      0         
_________________________________________________________________
time_distributed_22 (TimeDis (None, 5, 64)             411456    
_________________________________________________________________
lstm_27 (LSTM)               (None, 2)                 536       
Total params: 411,992
Trainable params: 411,992
Non-trainable params: 0
_________________________________________________________________


In [138]:
Input_type = 'video' # 'embedding'

if Input_type == 'video':
    Inp = Input( shape=(m,n, channel), name="Vid_Frame" )
    x = Conv2D(100, (3,3), activation='relu', padding='same')(Inp)
    x = Flatten()(x)
    x = Dense(64, name='Vid_Frame_Code')(x)
    Conv_Model = Model(Inp ,x)
    ########################### Printing information ########################
#     Conv_Model.summary()
    # print_model_weights(Conv_Model)
    
    vid_input = Input(shape=(num_frames, m,n, channel), name="Video")
    encoded_frames = TimeDistributed(Conv_Model, name="Code_Seq")(vid_input)  
    x = LSTM(num_words, return_sequences=True)(encoded_frames)  
    # return_sequences=False : the output will be a vector
    x = TimeDistributed(Dense(1))(x)  
    Video_Captioning_Model = Model(inputs=vid_input, outputs=x)


# elif Input_type == 'embedding':
#     Inp = Input( shape=(1,num_features),name = "Input" )
#     x = LSTM(num_features, name = "RNN_1")(Inp) 
#     Video_Captioning_Model = Model(Inp, x)

########################### Printing information ########################
Video_Captioning_Model.summary()
# print_model_weights(Video_Captioning_Model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Video (InputLayer)           (None, 5, 10, 10, 3)      0         
_________________________________________________________________
Code_Seq (TimeDistributed)   (None, 5, 64)             642864    
_________________________________________________________________
lstm_28 (LSTM)               (None, 5, 5)              1400      
_________________________________________________________________
time_distributed_23 (TimeDis (None, 5, 1)              6         
Total params: 644,270
Trainable params: 644,270
Non-trainable params: 0
_________________________________________________________________


In [84]:
X = np.random.rand(5,num_frames, m,n,channel)
Video_Captioning_Model.predict(X)

array([[-0.10395203, -0.11779281],
       [-0.09989359, -0.13424562],
       [-0.07368369, -0.11910312],
       [-0.1194683 , -0.14203915],
       [-0.08906429, -0.06782857]], dtype=float32)

## Many_to_Many (Leonard_2)

In [144]:
## For 1 particular input
length = 5
# seq = np.random.rand(num_features*length)
seq = np.array([i/float(length) for i in range(length)])

n_mfcc = 1
num_features = n_mfcc

## Input 
# The input for LSTMs must be three dimensional.
# (None=length, feature, channel)
#    length     : duration of speech,text
#    channel    : 1
#    feature    : lvl of embedding
X = seq.reshape(length, num_features, 1)

## Output 
# (None=length, feature)
#    length     : duration of speech,text
#    channel    : 1
#    feature    : lvl of embedding
y = seq.reshape(length, num_features, 1)

print('X', np.shape(X))
print('y', np.shape(y))

X (5, 1, 1)
y (5, 1, 1)


In [146]:
length = 5
seq = np.array([i/float(length) for i in range(length)])
X = seq.reshape(1, length, 1); print('X',X)
y = seq.reshape(1, length, 1); print('y',y)

X [[[ 0. ]
  [ 0.2]
  [ 0.4]
  [ 0.6]
  [ 0.8]]]
y [[[ 0. ]
  [ 0.2]
  [ 0.4]
  [ 0.6]
  [ 0.8]]]


In [ ]:
LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', 
     recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, 
     recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
     recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)


## Sequential with TimeDistributed 

In [ ]:
# prepare sequence
length = 5
seq = np.array([i/float(length) for i in range(length)])
X = seq.reshape(1, length, 1)
y = seq.reshape(1, length, 1)
print('X', np.shape(X))
print('y', np.shape(y))
# define LSTM configuration
n_neurons = length
########################### Architecture ##################################
# create LSTM
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(length, 1), return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mean_squared_error', optimizer='adam')
########################### Printing information ########################
model.summary()
# print_model_weights(model)
################################# Training ##############################
n_batch = 1
n_epoch = 1
# train LSTM
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=1)
################################ Evaluation #############################
result = model.predict(X)
print('result\n', result[0].T)
print('y\n', y[0].T)

# Test